In [1]:
cd /content/drive/My Drive/Thesis/code/relation-prediction-2

/content/drive/My Drive/Thesis/code/relation-prediction-2


In [0]:
pip install torchkge --quiet

In [0]:
pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.4.0.html --quiet

In [0]:
%load_ext autoreload
%autoreload 

In [0]:
import importlib

In [0]:
from layers import SpecialSpmmFinal

In [0]:
import train 
importlib.reload(train)
from train import *

In [0]:
import torch
import torch.nn as nn 

from functools import reduce
from operator import mul

from torch_scatter import scatter
from embedding import EmbeddingMul2

In [0]:
kg_train, kg_test, kg_val = load_fb15k237()
args = Args(100, 200, 100, 2, 100, 100, 0.01, 10, 'cuda', 'sgd')

In [10]:
n_ent, n_rel = kg_train.n_ent, kg_train.n_rel
total_triplets = get_valid_triplets(kg_train, kg_test, kg_val)

Number of unique triplets: 620232


In [0]:
dataloader = DataLoader(kg_train, batch_size=args.batch_size, shuffle=False, pin_memory=cuda.is_available())
ent_embed, rel_embed = get_init_embed()

In [0]:
class KGLayer(nn.Module):
    def __init__(self, n_entities, n_relations, ent_embed, rel_embed, in_dim, out_dim, concat=True, device="gpu"):
        super(KGLayer, self).__init__()

        self.n_entities = n_entities
        self.n_relations = n_relations
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.device = device

        self.a = nn.Linear(3 * in_dim, out_dim).to(device)
        nn.init.xavier_normal_(self.a.weight.data, gain=1.414)

        self.concat = concat

        self.a_2 = nn.Linear(out_dim, 1).to(device)
        nn.init.xavier_normal_(self.a_2.weight.data, gain=1.414)

        self.special_spmm_final = SpecialSpmmFinal()

        self.ent_embed = EmbeddingMul2(n_entities, in_dim, ent_embed, True, torch.device(device))
        self.rel_embed = EmbeddingMul2(n_relations, in_dim, rel_embed, True, torch.device(device))
    
    def forward(self, triplets):
        triplets = triplets.to(self.device)

        N = self.n_entities

        h = torch.cat((
            self.ent_embed(triplets[:, 0].unsqueeze(0)),
            self.ent_embed(triplets[:, 1].unsqueeze(0)),
            self.rel_embed(triplets[:, 2].unsqueeze(0))
        ), dim=1)
        c = self.a(h)
        b = F.leaky_relu(self.a_2(c))
        e_b = torch.exp(b)

        temp = triplets.t()
        edges = torch.stack([temp[0], temp[1]])

        e_b_sum = self.special_spmm_final(edges, e_b, N, e_b.shape[0], 1)
        temp1 = e_b * c

        h_sum = self.special_spmm_final(edges, temp1,  N, e_b.shape[0], self.out_dim)

        hs = h_sum
        ebs = e_b_sum
        ebs[ebs == 0] = 1e-12


        h_ent = hs / ebs

        index = triplets[:, 2]
        h_rel = scatter(temp1, index=index, dim=0, reduce="mean")

        if self.concat:
            return F.elu(h_ent), F.elu(h_rel)
        else:
            return h_ent, h_rel

In [0]:
model = KGLayer(n_ent, n_rel, ent_embed, rel_embed, 100, 100, True, "cuda")

In [0]:
optimizer = SGD(model.parameters(), lr=args.lr)

In [17]:
losses = []
for batch in dataloader:
    triplets = torch.stack(batch)
    triplets, labels, nodes, edges = negative_sampling(triplets, n_ent, args.negative_rate)
    triplets, labels = triplets.to(args.device), labels.to(args.device)

    model.zero_grad()

    # start = time.time()
    model.train()
    ent_embed_, rel_embed_ = model(triplets)
    loss = loss_func2(triplets, args.negative_rate, ent_embed_, rel_embed_, device="cuda")
    loss.backward()
    optimizer.step()

    losses.append(loss.item)
    print(loss.item())

4.647401809692383
4.754186630249023
4.620856285095215
4.535454750061035
4.677703380584717
4.554238796234131
4.570462226867676
4.573025703430176
4.610930919647217
4.538930892944336
4.308132171630859
4.6602091789245605
4.366220474243164
4.419582366943359
4.8886260986328125
4.521350860595703
4.469603538513184
4.760995864868164
4.6347784996032715
4.611804008483887
4.38735818862915
4.445220947265625
4.4584784507751465
4.676264762878418


RuntimeError: ignored